In [3]:
import threading
from collections import defaultdict, Counter

class General(threading.Thread):
    def __init__(self, general_id, num_generals, default_decision, traitor=False):
        super().__init__()
        self.general_id = general_id
        self.num_generals = num_generals
        self.traitor = traitor
        self.default_decision = default_decision
        self.decision = self.opposite_decision() if traitor else default_decision
        self.received_messages = defaultdict(list)
        self.final_decision = None

    def opposite_decision(self):
        return "retreat" if self.default_decision == "attack" else "attack"

    def run(self):
        for round_num in range(self.num_generals - 1):
            self.broadcast_decision(round_num)
        self.final_decision = self.consensus()
        print(f"General {self.general_id} (Is traitor?: {self.traitor}) --> {self.final_decision}")

    def broadcast_decision(self, round_num):
        for i in range(self.num_generals):
            if i != self.general_id:
                decision = self.opposite_decision() if self.traitor else self.decision
                self.received_messages[i].append(decision)

    def consensus(self):
        # count majority decision in final round
        final_decisions = [msg[-1] for msg in self.received_messages.values()]
        return max(set(final_decisions), key=final_decisions.count)

def final_conclusion(generals):
    decisions = [general.final_decision for general in generals]
    majority_decision, count = Counter(decisions).most_common(1)[0]
    print(f"\nFinal Conclusion:  {majority_decision.upper()}.\n")
    
    # list traitors if final decision differs from majority
    traitors = [general.general_id for general in generals if general.traitor]
    print(f"Traitors: {traitors if traitors else 'none detected'}")


num_generals = 5
num_traitors = 2
default_decision = "attack"  # default decision for all generals
generals = [
    General(general_id=i, num_generals=num_generals, default_decision=default_decision, traitor=(i < num_traitors))
    for i in range(num_generals)
]

for general in generals:
    general.start()

for general in generals:
    general.join()

final_conclusion(generals)


General 0 (Is traitor?: True) --> retreat
General 1 (Is traitor?: True) --> retreat
General 2 (Is traitor?: False) --> attack
General 3 (Is traitor?: False) --> attack
General 4 (Is traitor?: False) --> attack

Final Conclusion:  ATTACK.

Traitors: [0, 1]
